<a href="https://colab.research.google.com/github/LiBob0709/all-in-rag/blob/main/05LangChain%201.0-RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install -U langchain langchain-openai langchain-community pypdf python-docx faiss-cpu chroma sentence-transformers

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.3/328.3 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.0/253.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.2 MB/s eta 0:00:00
  Created wheel for chroma: filename=Chroma-0.2.0-py3-none-any.whl size=7095 sha256=3e0820250eca9523acad7b47c51e050b0cf5e4f8520902711f50082d1a53b644
  Stored in directory

In [5]:
from google.colab import userdata
API_KEY = userdata.get('DEEPSEEK_API_KEY')

# 加载不同格式文档

In [3]:
"""
文档加载示例
"""

from langchain_community.document_loaders import (
    PyPDFLoader,        # PDF
    Docx2txtLoader,     # Word
    TextLoader,         # 纯文本
    DirectoryLoader     # 批量加载
)

# 加载 PDF
pdf_loader = PyPDFLoader("company_report.pdf")
pdf_docs = pdf_loader.load()

print(f"加载了 {len(pdf_docs)} 页 PDF")
print(f"第一页内容预览: {pdf_docs[0].page_content[:100]}...")

# 加载 Word
docx_loader = Docx2txtLoader("policy.docx")
docx_docs = docx_loader.load()

# 加载文本文件
txt_loader = TextLoader("readme.txt", encoding="utf-8")
txt_docs = txt_loader.load()

# 批量加载目录下所有文档
dir_loader = DirectoryLoader(
    "docs/",
    glob="**/*.txt",  # 匹配所有 .txt 文件
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"}
)
all_docs = dir_loader.load()

print(f"目录下共加载 {len(all_docs)} 个文档")

ValueError: File path company_report.pdf is not a valid file or url

# Document 对象结构

In [ ]:
from langchain_core.documents import Document

doc = Document(
    page_content="这是文档内容...",
    metadata={
        "source": "report.pdf",
        "page": 1,
        "author": "张三",
        "date": "2024-01-15"
    }
)

print(doc.page_content)  # 内容
print(doc.metadata)      # 元数据

## 文本分块（Chunking）

###  为什么需要分块？

原因：

1. **长文档问题**：模型有 token 限制
2. **检索精度**：小块文本更容易匹配
3. **成本控制**：减少不必要的 token 消耗

In [ ]:
from langchain_text_splitters  import (
    RecursiveCharacterTextSplitter,  # 递归分块（推荐）
    CharacterTextSplitter,           # 字符分块
    TokenTextSplitter               # Token 分块
)

# 策略1：递归字符分块（推荐）
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,        # 每块最大字符数
    chunk_overlap=50,      # 块之间重叠字符数
    separators=["\n\n", "\n", "。", "！", "？", " ", ""]  # 分隔符优先级
)

text = """
第一段内容...

第二段内容...

第三段内容...
"""

chunks = splitter.split_text(text)
print(f"分成 {len(chunks)} 块")

# 策略2：分割文档对象
from langchain_core.documents import Document

docs = [
    Document(page_content="长文档内容...", metadata={"source": "doc1.txt"}),
    Document(page_content="另一个文档...", metadata={"source": "doc2.txt"})
]

split_docs = splitter.split_documents(docs)
print(f"分成 {len(split_docs)} 个文档块")

# 每个块保留原始元数据
for doc in split_docs[:3]:
    print(f"来源: {doc.metadata['source']}")
    print(f"内容: {doc.page_content[:50]}...\n")

# 中文分块的技巧

In [ ]:
# 中文专用分块器
chinese_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=[
        "\n\n",      # 段落
        "\n",        # 换行
        "。",        # 句号
        "！",        # 感叹号
        "？",        # 问号
        "；",        # 分号
        "，",        # 逗号
        " ",         # 空格
        ""          # 字符
    ],
    length_function=len  # 使用字符数
)

# 分块参数选择
| 参数              | 推荐值                | 说明               |
| ----------------- | --------------------- | ------------------ |
| **chunk_size**    | 300-1000 字符         | 中文约 150-500 字  |
| **chunk_overlap** | 10%-20% 的 chunk_size | 避免语义断裂       |
| **separators**    | 按语义层次            | 优先段落，其次句子 |

In [ ]:
# 短文本（FAQ、问答）
faq_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=20
)

# 长文档（论文、报告）
report_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

# 代码文档
code_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=50,
    separators=["\n\nclass ", "\n\ndef ", "\n\n", "\n", " "]
)

Embedding 将文本转换为**高维向量**（数字数组）：




```
文本: "人工智能发展迅速"
      ↓
向量: [0.23, -0.45, 0.67, ..., 0.12]  # 通常 768 或 1536 维
```
**相似文本的向量在空间中距离较近：**


```
"人工智能" → [0.2, 0.8, ...]
"AI技术"   → [0.3, 0.7, ...]  ← 相似，距离近

"今天天气" → [0.9, 0.1, ...]  ← 不相似，距离远
```



In [8]:
"""
Embedding 示例
"""

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")




# 向量化单个文本
text = "人工智能正在改变世界"
vector = embeddings.embed_query(text)

print(f"向量维度: {len(vector)}")
print(f"向量前5个值: {vector[:5]}")

# 批量向量化
texts = [
    "机器学习是人工智能的分支",
    "深度学习使用神经网络",
    "今天天气不错"
]

vectors = embeddings.embed_documents(texts)
print(f"批量向量化 {len(vectors)} 个文本")
# 计算相似度
import numpy as np

def cosine_similarity(vec1, vec2):
    """计算余弦相似度"""
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

vec1 = embeddings.embed_query("人工智能")
vec2 = embeddings.embed_query("AI技术")
vec3 = embeddings.embed_query("今天天气")

print(f"\n相似度:")
print(f"'人工智能' vs 'AI技术': {cosine_similarity(vec1, vec2):.4f}")
print(f"'人工智能' vs '今天天气': {cosine_similarity(vec1, vec3):.4f}")

向量维度: 1024
向量前5个值: [-0.017258409410715103, 0.020045343786478043, -0.027274254709482193, -0.006901141721755266, -0.043919552117586136]
批量向量化 3 个文本

相似度:
'人工智能' vs 'AI技术': 0.8056
'人工智能' vs '今天天气': 0.5274


In [18]:
"""
FAISS 向量数据库示例
"""

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.documents import Document

# 初始化 Embedding
from langchain_openai import OpenAIEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

# 准备文档
docs = [
    Document(page_content="LangChain 是一个 AI 应用开发框架", metadata={"source": "doc1"}),
    Document(page_content="Python 是一种编程语言", metadata={"source": "doc2"}),
    Document(page_content="机器学习需要大量数据", metadata={"source": "doc3"}),
    Document(page_content="深度学习使用神经网络", metadata={"source": "doc4"}),
    Document(page_content="今天天气很好，适合外出", metadata={"source": "doc5"}),
    Document(page_content="人工智能是未来的趋势", metadata={"source": "doc6"}),
    Document(page_content="hhhhh", metadata={"source": "doc7"}),
    Document(page_content="深度学习是机器学习的子集", metadata={"source": "doc8"}),
]

# 创建向量数据库
vectorstore = FAISS.from_documents(docs, embeddings)

print("✅ 向量数据库创建成功")

# 相似度搜索
query = "haha"
results = vectorstore.similarity_search(query, k=3)

print(f"\n查询: {query}")
print("最相关的 3 个文档:\n")

for i, doc in enumerate(results, 1):
    print(f"{i}. {doc.page_content}")
    print(f"   来源: {doc.metadata['source']}\n")

# 带相似度分数的搜索
results_with_scores = vectorstore.similarity_search_with_score(query, k=3)

print("带分数的结果:")
for doc, score in results_with_scores:
    print(f"分数 {score:.4f}: {doc.page_content}")

# 保存和加载
vectorstore.save_local("faiss_index")
print("\n✅ 向量数据库已保存")

# 加载
loaded_vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)
print("✅ 向量数据库已加载")

✅ 向量数据库创建成功

查询: haha
最相关的 3 个文档:

1. hhhhh
   来源: doc7

2. 今天天气很好，适合外出
   来源: doc5

3. 人工智能是未来的趋势
   来源: doc6

带分数的结果:
分数 0.4867: hhhhh
分数 1.0897: 今天天气很好，适合外出
分数 1.1813: 人工智能是未来的趋势

✅ 向量数据库已保存
✅ 向量数据库已加载


# 构建完整的RAG系统

In [22]:
"""
完整的 RAG 系统
"""

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# 1. 准备向量数据库（假设已有）
from langchain_openai import OpenAIEmbeddings
embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")

# 加载或创建向量数据库
vectorstore = FAISS.load_local(
    "faiss_index",
    embeddings,
    allow_dangerous_deserialization=True
)

# 2. 创建检索器
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}  # 返回前 3 个最相关文档
)

# 3. 创建提示词模板
prompt = ChatPromptTemplate.from_template("""你是一个问答助手。请基于以下上下文回答问题。

上下文：
{context}

问题：{question}

要求：
1. 仅基于上下文回答，不要编造信息
2. 如果上下文中没有相关信息，明确告知用户
3. 回答要准确、简洁

答案：""")

# 4. 创建模型
model = init_chat_model(
    "deepseek-chat",
    model_provider="openai",
    base_url="https://api.deepseek.com",
    api_key= API_KEY,
    temperature=0.0
)

# 5. 构建 RAG Chain
def format_docs(docs):
    """格式化检索到的文档"""
    return "\n\n".join([f"文档{i+1}:\n{doc.page_content}" for i, doc in enumerate(docs)])

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# 6. 使用
question = "今天天气如何？"
answer = rag_chain.invoke(question)

print(f"问题: {question}")
print(f"答案: {answer}")

问题: 今天天气如何？
答案: 根据上下文，文档1中提到“今天天气很好，适合外出”，因此今天天气很好。


# 实战项目 - 企业知识库问答系统
企业知识库问答系统


```
├── 文档处理模块
│   ├── 文档加载
│   ├── 文本分块
│   └── 向量化
├── 检索模块
│   ├── 向量数据库
│   ├── 相似度搜索
│   └── 结果重排序
├── 问答模块
│   ├── 提示词生成
│   ├── LLM 调用
│   └── 答案生成
└── 管理模块
    ├── 文档管理
    ├── 数据库维护
    └── 查询日志
```




In [24]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from typing import List
from langchain_core.documents import Document
import json
from datetime import datetime
from langchain_openai import OpenAIEmbeddings

load_dotenv()

class KnowledgeBase:
    """企业知识库系统"""

    def __init__(
        self,
        docs_directory: str = "./knowledge_docs",
        index_directory: str = "./knowledge_index",
        embedding_model: str = "BAAI/bge-m3"
    ):
        self.docs_directory = docs_directory
        self.index_directory = index_directory

        # 初始化 Embedding
        print("🔄 正在加载 Embedding 模型...")
        self.embeddings = HuggingFaceEmbeddings(model_name = "BAAI/bge-m3")
        print("✅ Embedding 模型加载完成")

        # 文本分块器
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            separators=["\n\n", "\n", "。", "！", "？", "；", "，", " ", ""]
        )

        # 向量数据库
        self.vectorstore = None

        # LLM
        self.model = init_chat_model(
            "deepseek-chat", # Corrected typo from deepseek-caht to deepseek-chat
            model_provider="openai",
            base_url="https://api.deepseek.com",
            api_key= API_KEY,
            temperature=0.0
        )

        # 查询历史
        self.query_log = []

    def build_index(self):
        """构建索引"""
        print("\n📚 开始构建知识库索引...")

        # 1. 加载文档
        print("🔄 正在加载文档...")
        loader = DirectoryLoader(
            self.docs_directory,
            glob="**/*.txt",
            loader_cls=TextLoader,
            loader_kwargs={"encoding": "utf-8"},
            show_progress=True
        )

        try:
            docs = loader.load()
            print(f"✅ 成功加载 {len(docs)} 个文档")
        except Exception as e:
            print(f"❌ 加载文档失败: {e}")
            return False

        if not docs:
            print("⚠️  没有找到文档，请确保文档目录中有 .txt 文件")
            return False

        # 2. 文本分块
        print("🔄 正在分块...")
        split_docs = self.text_splitter.split_documents(docs)
        print(f"✅ 分块完成，共 {len(split_docs)} 个文本块")

        # 3. 创建向量数据库
        print("🔄 正在向量化并建立索引...")
        self.vectorstore = FAISS.from_documents(split_docs, self.embeddings)
        print("✅ 向量数据库创建完成")

        # 4. 保存索引
        os.makedirs(self.index_directory, exist_ok=True)
        self.vectorstore.save_local(self.index_directory)
        print(f"✅ 索引已保存到 {self.index_directory}")

        return True

    def load_index(self):
        """加载已有索引"""
        print("🔄 正在加载索引...")
        try:
            self.vectorstore = FAISS.load_local(
                self.index_directory,
                self.embeddings,
                allow_dangerous_deserialization=True
            )
            print("✅ 索引加载成功")
            return True
        except Exception as e:
            print(f"❌ 加载索引失败: {e}")
            return False

    def query(self, question: str, k: int = 3) -> dict:
        """查询知识库"""
        if not self.vectorstore:
            return {"error": "知识库未初始化"}

        # 1. 检索相关文档
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": k}
        )

        docs = retriever.invoke(question)

        # 2. 构建提示词
        prompt = ChatPromptTemplate.from_template("""你是企业知识库问答助手。

          请基于以下文档片段回答问题。

          文档片段：
          {context}

          用户问题：{question}

          回答要求：
          1. 仅基于提供的文档内容回答
          2. 如果文档中没有相关信息，明确告知用户
          3. 回答要准确、完整、易懂
          4. 如果答案来自多个文档片段，请综合回答

          答案：""")

        # 3. 格式化文档
        def format_docs(docs):
            return "\n\n---\n\n".join([
                f"【文档 {i+1}】\n来源: {doc.metadata.get('source', '未知')}\n内容: {doc.page_content}"
                for i, doc in enumerate(docs)
            ])

        # 4. 构建 RAG Chain
        rag_chain = (
            {"context": lambda x: format_docs(docs), "question": RunnablePassthrough()}
            | prompt
            | self.model
            | StrOutputParser()
        )

        # 5. 生成答案
        answer = rag_chain.invoke(question)

        # 6. 记录查询
        self.query_log.append({
            "question": question,
            "answer": answer,
            "sources": [doc.metadata.get('source', '未知') for doc in docs],
            "timestamp": datetime.now().isoformat()
        })

        return {
            "question": question,
            "answer": answer,
            "sources": docs,
            "source_count": len(docs)
        }

    def export_logs(self, filename: str = "query_log.json"):
        """导出查询日志"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.query_log, f, ensure_ascii=False, indent=2)
        print(f"✅ 查询日志已导出到 {filename}")

def create_sample_docs(directory: str = "./knowledge_docs"):
    """创建示例文档"""
    os.makedirs(directory, exist_ok=True)

    docs = {
        "company_intro.txt": """公司简介\n\n        我们是一家专注于人工智能技术研发的科技公司，成立于2020年。\n\n        公司使命：让AI技术普惠每个人。\n\n        主要产品：\n        1. AI 开发平台\n        2. 智能客服系统\n        3. 知识库问答系统\n\n        公司规模：员工200人，其中研发人员占70%。\n\n        公司地址：北京市海淀区中关村软件园。""",

                "product_ai_platform.txt": """AI 开发平台\n\n        产品名称：CloudAI Platform\n\n        功能特性：\n        1. 模型训练：支持多种深度学习框架\n        2. 模型部署：一键部署到云端\n        3. API 接口：提供 RESTful API\n        4. 监控告警：实时监控模型性能\n\n        技术优势：\n        - 支持分布式训练\n        - 自动调参\n        - 模型版本管理\n        - A/B 测试\n\n        定价方案：\n        - 免费版：每月 1000 次 API 调用\n        - 专业版：每月 ¥999，10万次调用\n        - 企业版：定制化，联系销售""",

                "customer_service.txt": """智能客服系统\n\n        产品名称：SmartCS\n\n        核心功能：\n        1. 多轮对话：支持复杂的多轮对话\n        2. 知识库：集成企业知识库\n        3. 工单系统：自动创建工单\n        4. 数据分析：客服数据可视化\n
        适用场景：\n        - 电商客服\n        - 银行客服\n        - 政府服务热线\n\n        接入方式：\n        - 网页插件\n        - 移信公众号\n        - 小程序\n        - APP SDK\n\n        客户案例：\n        某电商平台接入后，客服效率提升 60%，客户满意度提高 35%。""",

                "hr_benefits.txt": """员工福利\n\n        我司为员工提供完善的福利体系：\n\n        薪酬福利：\n        1. 具有竞争力的薪资\n        2. 年终奖金（13-16薪）\n        3. 股票期权\n\n        保险福利：\n        1. 五险一金\n        2. 补充商业保险\n        3. 年度体检\n\n        假期福利：\n        1. 法定节假日\n        2. 带薪年假（10-20天）\n        3. 病假、婚假、产假等\n\n        其他福利：\n        1. 下午茶\n        2. 健身房\n        3. 员工旅游\n        4. 节日礼品\n        5. 生日派对""",

                "faq.txt": """常见问题\n\n        Q1: 如何联系技术支持？\n        A1: 发送邮件到 support@company.com，或拨打热线 400-123-4567。\n\n        Q2: API 调用限制是多少？\n        A2: 免费版每月1000次，专业版10万次，企业版可定制。\n\n        Q3: 支持哪些支付方式？\n        A3: 支持支付宝、微信支付、对公转账。\n\n        Q4: 数据安全如何保障？\n        A4: 采用银行级加密，通过 ISO27001 认证，数据中心设在国内。\n\n        Q5: 是否支持私有化部署？\n        A5: 企业版支持私有化部署，具体方案请联系销售。"""
    }

    for filename, content in docs.items():
        filepath = os.path.join(directory, filename)
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)

    print(f"✅ 已创建 {len(docs)} 个示例文档到 {directory}")

def main():
    """主程序"""
    print("=" * 70)
    print("📚 企业知识库问答系统（LangChain 1.0）")
    print("=" * 70)

    kb = KnowledgeBase()

    # 检查是否有已存在的索引
    if os.path.exists(kb.index_directory):
        print("\n发现已有索引")
        choice = input("是否使用已有索引？(y/n): ").strip().lower()
        if choice == 'y':
            if not kb.load_index():
                print("加载失败，需要重新构建")
                create_sample_docs()
                kb.build_index()
        else:
            create_sample_docs()
            kb.build_index()
    else:
        print("\n未找到索引，开始构建...")
        create_sample_docs()
        if not kb.build_index():
            print("❌ 构建失败，程序退出")
            return

    # 示例查询
    print("\n" + "=" * 70)
    print("📝 示例查询")
    print("=" * 70)

    test_questions = [
        "公司主要产品有哪些？",
        "AI 开发平台的定价是多少？",
        "员工有哪些福利？",
        "如何联系技术支持？"
    ]

    for question in test_questions:
        print(f"\n❓ {question}")
        result = kb.query(question)

        print(f"💡 {result['answer']}")
        print(f"📄 参考文档: {', '.join([doc.metadata['source'] for doc in result['sources']])}")
        print("-" * 70)

    # 交互模式
    print("\n" + "=" * 70)
    print("💬 进入交互模式（输入 'quit' 退出，'export' 导出日志）")
    print("=" * 70)

    while True:
        question = input("\n❓ 请输入问题: ").strip()

        if question.lower() == 'quit':
            kb.export_logs()
            print("再见！👋")
            break

        if question.lower() == 'export':
            kb.export_logs()
            continue

        if not question:
            continue

        result = kb.query(question)

        print(f"\n💡 答案:\n{result['answer']}")
        print(f"\n📚 参考来源:")
        for i, doc in enumerate(result['sources'], 1):
            print(f"  {i}. {doc.metadata['source']}")
            print(f"     内容片段: {doc.page_content[:100]}...")

if __name__ == "__main__":
    main()

📚 企业知识库问答系统（LangChain 1.0）
🔄 正在加载 Embedding 模型...
✅ Embedding 模型加载完成

发现已有索引
是否使用已有索引？(y/n): y
🔄 正在加载索引...
✅ 索引加载成功

📝 示例查询

❓ 公司主要产品有哪些？
💡 根据提供的文档内容，公司主要产品如下：

1. **AI 开发平台**
2. **智能客服系统**（产品名称为 SmartCS）
3. **知识库问答系统**

这些信息来源于【文档 1】的公司简介部分。
📄 参考文档: knowledge_docs/company_intro.txt, knowledge_docs/customer_service.txt, knowledge_docs/hr_benefits.txt
----------------------------------------------------------------------

❓ AI 开发平台的定价是多少？
💡 根据提供的文档内容，AI 开发平台（CloudAI Platform）的定价方案如下：

- **免费版**：每月提供 1000 次 API 调用。
- **专业版**：每月 ¥999，提供 10 万次 API 调用。
- **企业版**：为定制化方案，需联系销售获取具体信息。

以上信息综合自文档 1（产品介绍）和文档 3（常见问题）。
📄 参考文档: knowledge_docs/product_ai_platform.txt, knowledge_docs/company_intro.txt, knowledge_docs/faq.txt
----------------------------------------------------------------------

❓ 员工有哪些福利？
💡 根据提供的文档内容，员工福利信息主要来源于【文档 1】。具体如下：

**薪酬福利：**
1. 具有竞争力的薪资
2. 年终奖金（13-16薪）
3. 股票期权

**保险福利：**
1. 五险一金
2. 补充商业保险
3. 年度体检

**假期福利：**
1. 法定节假日
2. 带薪年假（10-20天）
3. 病假、婚假、产假等

**其他福利：**
1. 下午茶
2. 健身房
3. 

In [ ]:
"""
实战项目：企业知识库问答系统（LangChain 1.0）

功能：
- 文档批量导入
- 智能分块
- 向量检索
- RAG 问答
- 答案溯源
"""

import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.chat_models import init_chat_model
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from typing import List
from langchain_core.documents import Document
import json
from datetime import datetime
from langchain_openai import OpenAIEmbeddings

load_dotenv()

class KnowledgeBase:
    """企业知识库系统"""

    def __init__(
        self,
        docs_directory: str = "./knowledge_docs",
        index_directory: str = "./knowledge_index",
        embedding_model: str = "BAAI/bge-m3"
    ):
        self.docs_directory = docs_directory
        self.index_directory = index_directory

        # 初始化 Embedding
        print("🔄 正在加载 Embedding 模型...")
        self.embeddings =  OpenAIEmbeddings(
          base_url="https://api.siliconflow.cn/v1",
          api_key=API_KEY,
          model="BAAI/bge-m3"
        )
        print("✅ Embedding 模型加载完成")

        # 文本分块器
        self.text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=500,
            chunk_overlap=50,
            separators=["\n\n", "\n", "。", "！", "？", "；", "，", " ", ""]
        )

        # 向量数据库
        self.vectorstore = None

        # LLM
        self.model = init_chat_model(
            "Qwen/Qwen3-8B",
            model_provider="openai",
            base_url="https://api.siliconflow.cn/v1",
            api_key= API_KEY,
            temperature=0.0
        )

        # 查询历史
        self.query_log = []

    def build_index(self):
        """构建索引"""
        print("\n📚 开始构建知识库索引...")

        # 1. 加载文档
        print("🔄 正在加载文档...")
        loader = DirectoryLoader(
            self.docs_directory,
            glob="**/*.txt",
            loader_cls=TextLoader,
            loader_kwargs={"encoding": "utf-8"},
            show_progress=True
        )

        try:
            docs = loader.load()
            print(f"✅ 成功加载 {len(docs)} 个文档")
        except Exception as e:
            print(f"❌ 加载文档失败: {e}")
            return False

        if not docs:
            print("⚠️  没有找到文档，请确保文档目录中有 .txt 文件")
            return False

        # 2. 文本分块
        print("🔄 正在分块...")
        split_docs = self.text_splitter.split_documents(docs)
        print(f"✅ 分块完成，共 {len(split_docs)} 个文本块")

        # 3. 创建向量数据库
        print("🔄 正在向量化并建立索引...")
        self.vectorstore = FAISS.from_documents(split_docs, self.embeddings)
        print("✅ 向量数据库创建完成")

        # 4. 保存索引
        os.makedirs(self.index_directory, exist_ok=True)
        self.vectorstore.save_local(self.index_directory)
        print(f"✅ 索引已保存到 {self.index_directory}")

        return True

    def load_index(self):
        """加载已有索引"""
        print("🔄 正在加载索引...")
        try:
            self.vectorstore = FAISS.load_local(
                self.index_directory,
                self.embeddings,
                allow_dangerous_deserialization=True
            )
            print("✅ 索引加载成功")
            return True
        except Exception as e:
            print(f"❌ 加载索引失败: {e}")
            return False

    def query(self, question: str, k: int = 3) -> dict:
        """查询知识库"""
        if not self.vectorstore:
            return {"error": "知识库未初始化"}

        # 1. 检索相关文档
        retriever = self.vectorstore.as_retriever(
            search_type="similarity",
            search_kwargs={"k": k}
        )

        docs = retriever.invoke(question)

        # 2. 构建提示词
        prompt = ChatPromptTemplate.from_template("""你是企业知识库问答助手。

          请基于以下文档片段回答问题。

          文档片段：
          {context}

          用户问题：{question}

          回答要求：
          1. 仅基于提供的文档内容回答
          2. 如果文档中没有相关信息，明确告知用户
          3. 回答要准确、完整、易懂
          4. 如果答案来自多个文档片段，请综合回答

          答案：""")

        # 3. 格式化文档
        def format_docs(docs):
            return "\n\n---\n\n".join([
                f"【文档 {i+1}】\n来源: {doc.metadata.get('source', '未知')}\n内容: {doc.page_content}"
                for i, doc in enumerate(docs)
            ])

        # 4. 构建 RAG Chain
        rag_chain = (
            {"context": lambda x: format_docs(docs), "question": RunnablePassthrough()}
            | prompt
            | self.model
            | StrOutputParser()
        )

        # 5. 生成答案
        answer = rag_chain.invoke(question)

        # 6. 记录查询
        self.query_log.append({
            "question": question,
            "answer": answer,
            "sources": [doc.metadata.get('source', '未知') for doc in docs],
            "timestamp": datetime.now().isoformat()
        })

        return {
            "question": question,
            "answer": answer,
            "sources": docs,
            "source_count": len(docs)
        }

    def export_logs(self, filename: str = "query_log.json"):
        """导出查询日志"""
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(self.query_log, f, ensure_ascii=False, indent=2)
        print(f"✅ 查询日志已导出到 {filename}")

def create_sample_docs(directory: str = "./knowledge_docs"):
    """创建示例文档"""
    os.makedirs(directory, exist_ok=True)

    docs = {
        "company_intro.txt": """公司简介

        我们是一家专注于人工智能技术研发的科技公司，成立于2020年。

        公司使命：让AI技术普惠每个人。

        主要产品：
        1. AI 开发平台
        2. 智能客服系统
        3. 知识库问答系统

        公司规模：员工200人，其中研发人员占70%。

        公司地址：北京市海淀区中关村软件园。""",

                "product_ai_platform.txt": """AI 开发平台

        产品名称：CloudAI Platform

        功能特性：
        1. 模型训练：支持多种深度学习框架
        2. 模型部署：一键部署到云端
        3. API 接口：提供 RESTful API
        4. 监控告警：实时监控模型性能

        技术优势：
        - 支持分布式训练
        - 自动调参
        - 模型版本管理
        - A/B 测试

        定价方案：
        - 免费版：每月 1000 次 API 调用
        - 专业版：每月 ¥999，10万次调用
        - 企业版：定制化，联系销售""",

                "customer_service.txt": """智能客服系统

        产品名称：SmartCS

        核心功能：
        1. 多轮对话：支持复杂的多轮对话
        2. 知识库：集成企业知识库
        3. 工单系统：自动创建工单
        4. 数据分析：客服数据可视化

        适用场景：
        - 电商客服
        - 银行客服
        - 政府服务热线

        接入方式：
        - 网页插件
        - 移信公众号
        - 小程序
        - APP SDK

        客户案例：
        某电商平台接入后，客服效率提升 60%，客户满意度提高 35%。""",

                "hr_benefits.txt": """员工福利

        我司为员工提供完善的福利体系：

        薪酬福利：
        1. 具有竞争力的薪资
        2. 年终奖金（13-16薪）
        3. 股票期权

        保险福利：
        1. 五险一金
        2. 补充商业保险
        3. 年度体检

        假期福利：
        1. 法定节假日
        2. 带薪年假（10-20天）
        3. 病假、婚假、产假等

        其他福利：
        1. 下午茶
        2. 健身房
        3. 员工旅游
        4. 节日礼品
        5. 生日派对""",

                "faq.txt": """常见问题

        Q1: 如何联系技术支持？
        A1: 发送邮件到 support@company.com，或拨打热线 400-123-4567。

        Q2: API 调用限制是多少？
        A2: 免费版每月1000次，专业版10万次，企业版可定制。

        Q3: 支持哪些支付方式？
        A3: 支持支付宝、微信支付、对公转账。

        Q4: 数据安全如何保障？
        A4: 采用银行级加密，通过 ISO27001 认证，数据中心设在国内。

        Q5: 是否支持私有化部署？
        A5: 企业版支持私有化部署，具体方案请联系销售。"""
    }

    for filename, content in docs.items():
        filepath = os.path.join(directory, filename)
        with open(filepath, 'w', encoding='utf-8') as f:
            f.write(content)

    print(f"✅ 已创建 {len(docs)} 个示例文档到 {directory}")

def main():
    """主程序"""
    print("=" * 70)
    print("📚 企业知识库问答系统（LangChain 1.0）")
    print("=" * 70)

    kb = KnowledgeBase()

    # 检查是否有已存在的索引
    if os.path.exists(kb.index_directory):
        print("\n发现已有索引")
        choice = input("是否使用已有索引？(y/n): ").strip().lower()
        if choice == 'y':
            if not kb.load_index():
                print("加载失败，需要重新构建")
                create_sample_docs()
                kb.build_index()
        else:
            create_sample_docs()
            kb.build_index()
    else:
        print("\n未找到索引，开始构建...")
        create_sample_docs()
        if not kb.build_index():
            print("❌ 构建失败，程序退出")
            return

    # 示例查询
    print("\n" + "=" * 70)
    print("📝 示例查询")
    print("=" * 70)

    test_questions = [
        "公司主要产品有哪些？",
        "AI 开发平台的定价是多少？",
        "员工有哪些福利？",
        "如何联系技术支持？"
    ]

    for question in test_questions:
        print(f"\n❓ {question}")
        result = kb.query(question)

        print(f"💡 {result['answer']}")
        print(f"📄 参考文档: {', '.join([doc.metadata['source'] for doc in result['sources']])}")
        print("-" * 70)

    # 交互模式
    print("\n" + "=" * 70)
    print("💬 进入交互模式（输入 'quit' 退出，'export' 导出日志）")
    print("=" * 70)

    while True:
        question = input("\n❓ 请输入问题: ").strip()

        if question.lower() == 'quit':
            kb.export_logs()
            print("再见！👋")
            break

        if question.lower() == 'export':
            kb.export_logs()
            continue

        if not question:
            continue

        result = kb.query(question)

        print(f"\n💡 答案:\n{result['answer']}")
        print(f"\n📚 参考来源:")
        for i, doc in enumerate(result['sources'], 1):
            print(f"  {i}. {doc.metadata['source']}")
            print(f"     内容片段: {doc.page_content[:100]}...")

if __name__ == "__main__":
    main()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')